In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample


In [8]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 comment='#', sep=',')
    
    return df

In [9]:
def createData():
    df = readCsv('1_Cov.csv')
    meanPositive = readCsv('1_m_1.csv').dropna(axis=1, how='all').as_matrix()
    meanNegative = readCsv('1_m_0.csv').dropna(axis=1, how='all').as_matrix()
    meanPositive = meanPositive[0,:]
    meanNegative = meanNegative[0,:]
    
    cov = df.dropna(axis=1, how='all').as_matrix()
    
    distributionPositive = np.random.multivariate_normal(meanPositive, cov, 2000)
    distributionNegative = np.random.multivariate_normal(meanNegative, cov, 2000)
    
    distributionPositive = np.c_[distributionPositive, np.ones(distributionPositive.shape[0])]
    distributionNegative = np.c_[distributionNegative, np.zeros(distributionNegative.shape[0])]
#     print('Positive', distributionPositive)
#     print('Negative', distributionNegative)
    distribution = np.concatenate((distributionPositive, distributionNegative), axis=0)
    np.random.shuffle(distribution)
    distributionTest = distribution[0:1200]
    distributionTrain = distribution[1200:] 
    print(distribution)
    print(distributionTest)
    print(distributionTrain)
    print('original', distribution.shape)
    print('train', distributionTrain.shape)
    print('test', distributionTest.shape)
    return (distribution, distributionTest, distributionTrain)


In [10]:
def toCsv(fileName, npArray):
    
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')


In [11]:
def initializeDS1():
    fullDistribution, distributionTest, distributionTrain = createData()
    toCsv('1_train.csv', distributionTrain)
    toCsv('1_test.csv', distributionTest)
    toCsv('1.csv', fullDistribution)
initializeDS1()


[[ 0.34202437  1.35147519  2.52792192 ...,  1.07567635 -2.23509516  0.        ]
 [ 2.27473906  0.72331772  2.55903595 ..., -0.32376621  0.98610758  0.        ]
 [ 6.95724059  5.88878613  4.8385471  ...,  2.88931056  3.85766229  0.        ]
 ..., 
 [ 2.64886532  2.46943597  1.72987447 ...,  4.30558941  3.64403896  0.        ]
 [ 2.51985751  1.39122851  1.11322351 ...,  0.13196815  2.28450757  1.        ]
 [-1.88003678 -0.35812245 -2.13434339 ..., -1.15754924  2.11947785  1.        ]]
[[ 0.34202437  1.35147519  2.52792192 ...,  1.07567635 -2.23509516  0.        ]
 [ 2.27473906  0.72331772  2.55903595 ..., -0.32376621  0.98610758  0.        ]
 [ 6.95724059  5.88878613  4.8385471  ...,  2.88931056  3.85766229  0.        ]
 ..., 
 [ 0.22473106 -0.26968278  2.15060049 ...,  1.85207128 -1.38497024  1.        ]
 [ 1.76623179  4.0809592   3.7035515  ...,  2.83982552  2.94995842  0.        ]
 [ 1.06299512 -0.13768842  2.46645962 ...,  0.38812707 -0.52804695  0.        ]]
[[-0.08914377  1.1347193

In [12]:
def visualizeStuff():
    mean = [10, 10, 10]
    cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]] 
    x, y, z = np.random.multivariate_normal(mean, cov, 10000).T

    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(111, projection='3d')

    plt.scatter(x, y, z, c='red')
    plt.axis('equal')
    plt.show()
# visualizeStuff()

In [52]:
def runLDA():
    dfTrain = readCsv('1_train.csv')
    N, N1, N2, pi, U1, U2 = np.zeros(6)
    Y = dfTrain[20].as_matrix()
    dfTrain = dfTrain.iloc[:,0:20]
    S = np.zeros((20,20))
    S1 = np.zeros((20,20))
    S2 = np.zeros((20,20))
    N = len(dfTrain)
    N1 = np.count_nonzero(Y)
    N2 = N - N1
    pi = N1/N
    
    for i in range(0, len(dfTrain)):
        U1 = U1 + Y[i]*dfTrain.iloc[i,:].as_matrix()
        U2 = U2 + (1-Y[i])*dfTrain.iloc[i,:].as_matrix()
    U1 = U1/N1
    U2 = U2/N2
    
    for i in range(0, len(dfTrain)):
        xSubMean = np.zeros(20)
        if (Y[i]==1):
            xSubMean = np.subtract(dfTrain.iloc[0,:].as_matrix(), U1)   
            S1 = S1 + np.outer(xSubMean, xSubMean)
        elif (Y[i]==0):
            xSubMean = np.subtract(dfTrain.iloc[0,:].as_matrix(), U2)
            S2 = S2 + np.outer(xSubMean, xSubMean)
        
    S = np.add(S1, S2)
    S = np.divide(S, N)
    return (U1, U2, pi, S)

runLDA()


(array([ 1.92486305,  1.88574537,  1.91903303,  1.9023876 ,  1.87220534,
         1.92675551,  1.90384851,  1.93631247,  1.92383374,  1.88307188,
         1.93366121,  1.8939727 ,  1.8565985 ,  1.92555162,  1.90964703,
         1.86961943,  1.91901221,  1.90444018,  1.92062611,  1.86677443]),
 array([ 1.2578635 ,  1.33369734,  1.22781812,  1.24713771,  1.2745293 ,
         1.2742541 ,  1.29876692,  1.24321133,  1.23592936,  1.29741145,
         1.26109714,  1.26499492,  1.27306806,  1.24368497,  1.26658048,
         1.24846667,  1.20458703,  1.26859286,  1.28606356,  1.29473774]),
 0.5,
 array([[  2.93532603e+00,   8.90298118e-01,   5.89167105e-01,
           3.78424016e+00,   1.17119551e+00,   8.94692917e-01,
          -9.00455793e-02,   3.91132761e+00,   8.63244395e-01,
           2.41901571e+00,   2.06436796e+00,   3.67968614e+00,
           5.10836387e+00,   7.16803550e-01,  -5.15775810e-01,
           8.21158558e-01,   3.07741130e+00,   3.19499868e+00,
           1.32925014e+00,  

In [59]:
def linearDecisionBoundary():
    U1, U2, pi, S = runLDA()
    invS = np.linalg.inv(S)
    w = np.dot(invS, np.subtract(U1, U2))
    w0 = 0
    w0 = np.divide(np.dot(U1.T, np.dot(invS, U1)), -2)
    w0 = w0 + np.divide(np.dot(U2.T, np.dot(invS, U2)), 2)
    w0 = w0 + np.log(pi/(1-pi))
    
#     print(w)
    return (w, w0)

linearDecisionBoundary()

[ 0.06762695  1.42578125 -0.6015625  -0.08203125  0.59570312 -1.1953125
  1.6796875  -0.48828125  0.91796875 -0.51074219  0.52148438  0.17773438
  0.04296875  0.0859375   1.515625    1.3125      0.19750977 -0.66796875
  0.70605469  0.93847656]


(array([ 0.06762695,  1.42578125, -0.6015625 , -0.08203125,  0.59570312,
        -1.1953125 ,  1.6796875 , -0.48828125,  0.91796875, -0.51074219,
         0.52148438,  0.17773438,  0.04296875,  0.0859375 ,  1.515625  ,
         1.3125    ,  0.19750977, -0.66796875,  0.70605469,  0.93847656]),
 -10.58203125)

In [57]:
def testData():
    w, w0 = linearDecisionBoundary()
    dfTest = readCsv('1_test.csv')
    X = dfTest[20].as_matrix()
    dfTest = dfTest.iloc[:,0:20]
    classifier = 0
    print(w0)
#     for i in range(0, len(dfTest)):
#         probability = np.dot(w)
testData()

0.5


TypeError: 'NoneType' object is not iterable